In [1]:
import datetime as dt
import os
import pprint
import shutil
import json
import mimetypes


from fuzzywuzzy import process
import requests
from requests_html import HTMLSession
from requests import exceptions as reqexps

mimetypes.init()
mimes = mimetypes.MimeTypes()

pretty = pprint.PrettyPrinter(indent=4)

In [2]:
start = dt.datetime.now()
with open('data/input/books.txt', 'r') as f:
    bookText = f.read()

bookData = []

for line in bookText.split('\n'):
    if line.startswith('!!'):
        pass
    else:
        title, link = line.split('-||-')
        
        title = title.strip()
        link  = link.strip()
        img_title = title.replace(' ', '-')
        
        session = HTMLSession()
        
        # r = requests.get()
        r = session.get('https:'+link)

        try:
            # try try again to select the image (depending on type of link/page was saved 
            #   (usually audio book but variablity exists)
            
            main_image_elem = r.html.find('#main-image', first=True)
            if not main_image_elem:
                main_image_elem = r.html.find('#imgBlkFront', first=True)
            if not main_image_elem:
                main_image_elem = r.html.find('#ebooksImgBlkFront', first=True)            
            main_image_link = main_image_elem.attrs.get('src',None) 
            
            r = requests.get(main_image_link, stream=True)
            
            if r.status_code == 200:
                _, ext = os.path.splitext(r.url)
                with open('data/output/'+img_title + ext, 'wb') as f:
                    r.raw.decode_content = True
                    shutil.copyfileobj(r.raw, f) 

            bookData.append({'title': title, 
                             'link': link,
                             'main_image_link':main_image_link,
                             'img_title': img_title,
                             'image_path':img_title +'.'+ ext
                            })

        except AttributeError as e:
            print('http:'+link, title)
        except reqexps.InvalidSchema as e:
            print('http:'+link, title)

pretty.pprint(bookData[:3])
print(dt.datetime.now() - start)

ConnectionError: HTTPSConnectionPool(host='amzn.to', port=443): Max retries exceeded with url: /2EZCz3m (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x10ec5e198>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [3]:
with open('data/output/processedBooks.json','w') as fp:
    json.dump(bookData, fp)

In [4]:
# make it stick - , The Science of Successful Learning

others = """A Technique for Producing Ideas
Make it Stick
Serial Innovators: Firms That Change the World
Fighting Talk: Forty Maxims on War, Peace, and Strategy
Churchill: On The Power of Words
Only Humans Need Apply: Winners and Losers in the Age of Smart Machines
Boom Towns: Restoring the Urban American Dream
The Quantum Spy: A Thriller
Sam Walton: Made In America
Tribal Leadership: Leveraging Natural Groups to Build a Thriving Organization
The Inner Lives of Markets: How People Shape Them
Cathcher in the Rye
Berkshire Hathaway Letters to Shareholders
Confessions of an Advertising Man
The Intelligent Investor: The Definitive Book on Value Investing
The Wealth of Nations
The Adweek Copywriting Handbook: The Ultimate Guide to Writing Powerful Advertising and Marketing y
Ogilvy on Advertising
1776: David McCullough
The Signal And the Noise: Nate Silvers
Great Courses: The Art of Negotiating The Best Deal: Prof: Seth Freeman
Business Strategy: Spender
Thinking Fast and Thiking Slow: Daniel Kahneman
The Caves of Steel: Asimov
The Robots of Dawn: Asimov
Competitive Forces: Michael E Porter
Great Courses: Influence: Mastering Life's Most"""

In [5]:
filePathSet = """1776-mccoullough.jpg
Fighting-Talk-book-cover.jpg
berkshire-letters.jpg
boom-towns.jpg
business-strat.jpg
catcher-in-the-rye.jpg
The-Caves-Of-Steel-Asimov.jpg
churchill.jpg
Confessions-of-an-Advertising-Man.jpg
Copywriting-Handbook.jpg
Influence.jpg
The-Inner-Lives-of-Markets-How-People-Shape-Them.jpg
itelligent-investor.jpg
Sam-Walton-made-in-america.jpg
make-it-stick.jpg
Great-Courses-The-Art-of-Negotiating-The-Best-Deal.jpg
Ogilvy-on-Advertising.jpg
only-humans.jpg
porter-competitve-strategy.jpg
produce-ideas.jpg
quantum-spy.jpg
The-Robots-of-Dawn-Asimov.jpg
Serial-Innovators.jpg
The-Signal-the-Noise-Nate-Silvers.jpg
thinking-fast-thinking-slow.jpg
tribal-leadership.jpg
The-Wealth-of-Nations.jpg"""

In [6]:
otherData = others.split('\n')
otherjson = []

In [ ]:
for file in filePathSet.split():
    print(file," :: ", process.extract(file, otherData, limit=1))
    otherjson.append({ 'image_path': process.extract(file, otherData, limit=1))[0]
                        'img_title': 'The-Measure-of-a-Man,-A-Spiritual-Autobiography:-Sidney-Portier',
                        'link': '//amzn.to/2EZCz3m',
                        'main_image_link': 'https://images-na.ssl-images-amazon.com/images/I/51Re8bE%2BAxL._AA300_.jpg',
                        'title': 'The Measure of a Man, A Spiritual Autobiography: Sidney'})
            

In [ ]:
with open('data/output/manuallyProcessedBooks.json','w', newline='\n') as fp:
    json.dump(otherjson, fp)